In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
!pip install lap cython bbox

In [ ]:
!pip install ultralytics lap cython filterpy scipy
!pip install numpy==1.24.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=d01414b501d4cdc16fa57bcdc00a716519ea17da1188a0e79cdc0e66381f04d8
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requir

In [ ]:
from ultralytics import YOLO
import cv2

# Load model YOLOv8
model = YOLO("/content/best.pt")

# Video input
video_path = "/content/01.mp4"
cap = cv2.VideoCapture(video_path)

# Lấy thông tin video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/content/output_with_id.mp4", fourcc, fps, (width, height))

# Class mapping
id2label = {0: "standing", 1: "lying", 2: "eating"}
class_list = list(id2label.values())

# Ghi log thời gian eating của mỗi heo
eating_log = {}

# Tracking
results = model.track(
    source=video_path,
    conf=0.4,
    persist=True,
    stream=True,
    save=False
)

# Duyệt từng frame
for result in results:
    frame = result.orig_img.copy()
    boxes = result.boxes
    if boxes is None or boxes.cls is None or boxes.id is None:
        out.write(frame)
        continue

    class_ids = boxes.cls.tolist()
    track_ids = boxes.id.tolist()
    xyxy = boxes.xyxy.cpu().numpy()

    if not class_ids or not track_ids:
        out.write(frame)
        continue

    # Đếm số theo posture
    count_by_class = {label: 0 for label in class_list}
    for cls_id in class_ids:
        label = id2label[int(cls_id)]
        count_by_class[label] += 1

    # Tổng số heo dựa trên sum của các class
    total_pigs = sum(count_by_class.values())

    # Ghi nhận thời gian eating
    for i, track_id in enumerate(track_ids):
        class_id = int(class_ids[i])
        label = id2label[class_id]
        if label == "eating":
            id_int = int(track_id)
            eating_log[id_int] = eating_log.get(id_int, 0) + 1

    # Vẽ box + nhãn + ID
    for i, box in enumerate(xyxy):
        x1, y1, x2, y2 = map(int, box)
        class_id = int(class_ids[i])
        label = id2label[class_id]
        track_id = int(track_ids[i])
        text = f"{label} ID {track_id}"

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, text, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Hiển thị số lượng posture ở góc
    y_offset = 30
    cv2.putText(frame, "🐷 Pig Count:", (10, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

    for i, label in enumerate(class_list):
        count = count_by_class[label]
        text = f"{label}: {count}"
        cv2.putText(frame, text, (10, y_offset + 30 + i * 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Thêm dòng tổng số heo
    cv2.putText(frame, f"Total pigs: {total_pigs}",
                (10, y_offset + 30 + len(class_list) * 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 3)

    # Ghi ra video
    out.write(frame)

# Kết thúc video
cap.release()
out.release()

#  In tổng thời gian eating từng con heo
print("\n🐷 Tổng thời gian ăn của từng con heo:")
for id_, frame_count in sorted(eating_log.items(), key=lambda x: -x[1]):
    seconds = frame_count / fps
    print(f"→ Heo ID {id_}: {frame_count} frames ≈ {seconds:.2f} giây")

#  Phát hiện heo ăn quá 15 phút
abnormal_threshold = 3 * 60 * fps  # 15 phút = 900s * fps
print("\n Heo ăn quá 15 phút (bất thường):")
for id_, frame_count in sorted(eating_log.items(), key=lambda x: -x[1]):
    if frame_count > abnormal_threshold:
        seconds = frame_count / fps
        print(f" Heo ID {id_}: {frame_count} frames ≈ {seconds:.2f} giây 🛑")

print("\n✅ Đã tạo video: /content/output_with_id.mp4")



video 1/1 (frame 1/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 15.6ms
video 1/1 (frame 2/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 17.3ms
video 1/1 (frame 3/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 16.3ms
video 1/1 (frame 4/8999) /content/01.mp4: 384x640 7 lyings, 16.9ms
video 1/1 (frame 5/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 16.1ms
video 1/1 (frame 6/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 16.4ms
video 1/1 (frame 7/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 15.2ms
video 1/1 (frame 8/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 18.3ms
video 1/1 (frame 9/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 16.8ms
video 1/1 (frame 10/8999) /content/01.mp4: 384x640 1 standing, 7 lyings, 15.2ms
video 1/1 (frame 11/8999) /content/01.mp4: 384x640 6 lyings, 16.4ms
video 1/1 (frame 12/8999) /content/01.mp4: 384x640 1 standing, 6 lyings, 16.1ms
video 1/1 (frame 13/8999) /content/01.mp4: 384x640 1 standing, 6 lyings,

KeyboardInterrupt: 